In [0]:
from google.colab import files
data_to_load = files.upload()

In [0]:
from numpy import dot , array
from math import sqrt
import networkx as nx
from random import sample , randint
from functools import reduce

In [0]:
import numpy as np
import time
import matplotlib.pyplot as plt
import csv
import random
import pickle
from random import random , randint

In [0]:
def prod(List):
    return reduce((lambda x, y: x * y), List)
    #### class Service ####


class Service:

    # constructor

    def __init__(self, activity, responseTime, reliability, availability, price, matching=1):

        if isinstance(activity, int):
            self.__activity = activity
        else:
            raise Exception("activity must be of type : int")

        if isinstance(responseTime, (int, float)):
            self.__responseTime = responseTime
        else:
            raise Exception("responseTime must be of type : int or float ")

        if isinstance(reliability, (int, float)):
            self.__reliability = reliability
        else:
            raise Exception("reliability must be of type : int or float ")

        if isinstance(availability, (int, float)):
            self.__availability = availability
        else:
            raise Exception("availability must be of type : int or float ")

        if isinstance(price, (int, float)):
            self.__price = price
        else:
            raise Exception("price must be of type : int or float ")

        if isinstance(matching, (int, float)) and 0 < matching <= 1:
            self.__matching = matching
        else:
            raise Exception("matching must be between 0 and 1")

    # get attributs

    def getResponseTime(self):
        return self.__responseTime

    def getPrice(self):
        return self.__price

    def getReliability(self):
        return self.__reliability

    def getAvailability(self):
        return self.__availability

    def getMatching(self):
        return self.__matching

    def getActivity(self):
        return self.__activity

    #  Euclidean Distance

    def euclideanDist(self, service):

        dRes = self.__responseTime - service.getResponseTime()
        dPri = self.__price - service.getPrice()
        dRel = self.__reliability - service.getReliability()
        dAva = self.__availability - service.getAvailability()

        return sqrt(dRes ** 2 + dPri ** 2 + dRel ** 2 + dAva ** 2)

    ##### class compositionPlan #####


class CompositionPlan:

    # constructor

    # argument actGraph is a list of lists each list represents an arc [A1 , A2 , 0 or 1 or -1]
    def __init__(self, actGraph, candidates):
        self.G = nx.DiGraph()
        self.G.add_weighted_edges_from(actGraph)
        for act, candidate in enumerate(candidates):
            self.G.nodes[act]["service"] = sample(candidate, 1)[0]

    # Calcul Methods

    def cpResponseTime(self, rootAct=0):
        try:
            outgoing = list(self.G.successors(rootAct))  # outgoing arcs
            neighbor = outgoing[0]  # first neighbor
            type = self.G.edges[rootAct, neighbor]["weight"]
            if type == 0:
                # type = sequential
                return  self.G.nodes[rootAct]["service"].getResponseTime() + self.cpResponseTime(neighbor)
            elif type == -1:
                # type = conditional
                s = 0
                n = 0
                for arc in outgoing:
                    n += 1
                    s += self.cpResponseTime(neighbor)
                return (s / n) + self.G.nodes[rootAct]["service"].getResponseTime()
            elif type == 1:
                # type = parallel
                return self.G.nodes[rootAct]["service"].getResponseTime() + max([self.cpResponseTime(neighbor) for arc in outgoing])

        except:  # node with no destination
            return self.G.nodes[rootAct]["service"].getResponseTime()

    def cpPrice(self, rootAct=0):
        try:
            outgoing = list(self.G.successors(rootAct))  # outgoing arcs
            neighbor = outgoing[0]  # first neighbor
            type = self.G.edges[rootAct, neighbor]["weight"]

            if type == 0:
                # type = sequentials
                return self.G.nodes[rootAct]["service"].getPrice() + self.cpPrice(neighbor)

            elif type == -1:
                # type = conditional
                s = 0
                n = 0
                for arc in outgoing:
                    n += 1
                    s += self.cpPrice(neighbor)
                return (s / n) + self.G.nodes[rootAct]["service"].getPrice()
            elif type == 1:
                # type = parallel
                return self.G.nodes[rootAct]["service"].getPrice() + max(
                    [self.cpPrice(neighbor) for arc in outgoing])
        except:
            return self.G.nodes[rootAct]["service"].getPrice()



    def cpAvailability(self, rootAct=0):
        try:
            outgoing = list(self.G.successors(rootAct))  # outgoing arcs
            neighbor = outgoing[0]  # first neighbor
            type = self.G.edges[rootAct, neighbor]["weight"]
            if type == 0:
                # type = sequential
                return self.G[rootAct]["service"].getAvailability() * self.cpAvailability(neighbor)
            elif type == -1:
                # type = conditional
                s = 0
                n = 0
                for arc in outgoing:
                    n += 1
                    s += self.cpAvailability(neighbor)
                return (s / n) * self.G.nodes[rootAct]["service"].getAvailability()
            elif type == 1:
                # type = parallel
                return self.G.nodes[rootAct]["service"].getAvailability() * prod([self.cpAvailability()(neighbor) for arc in outgoing])
        except:  # node with no destination
            return self.G.nodes[rootAct]["service"].getAvailability()

    def cpReliability(self, rootAct=0):
        try:
            outgoing = list(self.G.successors(rootAct))  # outgoing arcs
            neighbor = outgoing[0]  # first neighbor
            type = self.G.edges[rootAct, neighbor]["weight"]
            if type == 0:
                # type = sequential
                return self.G[rootAct]["service"].getReliability() * self.cpReliability(neighbor)
            elif type == -1:
                # type = conditional
                s = 0
                n = 0
                for arc in outgoing:
                    n += 1
                    s += self.cpReliability(neighbor)
                return (s / n) * self.G.nodes[rootAct]["service"].getReliability()
            elif type == 1:
                # type = parallel
                return self.G.nodes[rootAct]["service"].getReliability() * prod([self.cpReliability()(neighbor) for arc in outgoing])
        except :  # node with no destination
            return self.G.nodes[rootAct]["service"].getReliability()

    # Quality of Service

    def globalQos(self, minQos, maxQos, constraints, weightList):  # weightList should be in order (Price,ResponseTime,Availability,Reliability)

        drt = constraints['responseTime'] - self.cpResponseTime()
        dpr = constraints['price'] - self.cpPrice()
        dav = self.cpAvailability() - constraints['availability']
        drel = self.cpReliability() - constraints['reliability']
        if drt and dpr and dav and drel:
            rt = (maxQos['responseTime'] - self.cpResponseTime()) / (maxQos['responseTime'] - minQos['responseTime'])
            pr = (maxQos['price'] - self.cpPrice()) / (maxQos['price'] - minQos['price'])
            av = (self.cpAvailability() - minQos['availability']) / (maxQos['availability'] - minQos['availability'])
            rel = (self.cpReliability() - minQos['reliability']) / (maxQos['reliability'] - minQos['reliability'])

            vect1 = array([rt, pr, av, rel])
            # weights
            vect2 = array(weightList)
            # vectorial product
            return dot(vect1, vect2)
        else:
            return -1

    # Matching degree
    def cpMatching(self, rootAct=0):
        n = self.G.number_of_nodes()
        try:
            outgoing = list(self.G.successors(rootAct))  # outgoing arcs
            s = sum([self.cpMatching(neighbor) for neighbor in outgoing])
            return s + (self.G.nodes[rootAct]["service"].getMatching() / n)
        except:
            return self.G.nodes[rootAct]["service"].getMatching() / n

    # Modifying composition plan by mutating a service

    def mutate(self, new):
        self.G.nodes[new.getActivity()]["service"] = new


# CROSSOVER

def crossover(Plan1, Plan2):
    actGraph = list(Plan1.G.edges.data("weight"))
    candidates = [[act[1]] for act in list(Plan1.G.nodes.data("service"))]
    child = CompositionPlan(actGraph, candidates)
    for act in child.G.nodes:  # Selecting services to mutate
        if randint(0, 1):  # 50 % chance of mutation
            child.G.nodes[act]["service"] = Plan2.G.nodes[act]["service"]
    return child

In [0]:
def getNeighbors(s, candidates):  # s is a service
    L = candidates[s.getActivity()]
    return sorted([neighbor for neighbor in L if neighbor != s], key=lambda x: s.euclideanDist(x))


# Fitness function
def f(cp, minQos, maxQos, constraints, weightList):
    return cp.globalQos(minQos, maxQos, constraints, weightList) + cp.cpMatching()


# SQ : condition for scouts , MCN : termination condition , SN : number of compositionPlans , p :probability
def ABCgenetic(actGraph, candidates, workers, onlookers, scouts, SQ, MCN, SN, minQos, maxQos, constraints,weightList):
    # initializing
    solutions = list()
    fitnessList = list()
    probabilityList = list(0 for i in range(SN))
    limit = list(0 for i in range(SN))
    best_fit = 0
    CP = 4 * MCN / 5  # changing point for scouts
    # solutions and fitness initializing
    for i in range(SN):
        while 1:
            sol = CompositionPlan(actGraph, candidates)
            fit = f(sol, minQos, maxQos, constraints, weightList)
            if fit:
                solutions.append(sol)
                fitnessList.append(fit)
                break

    # Algorithm
    for itera in range(1, MCN + 1):

        # working bees phase
        for bee in range(workers):
            i = randint(0, SN - 1)
            cp1 = solutions[i]  # cp is a composition plan
            cp2 = solutions[randint(0, SN - 1)]
            # Crossover
            child = crossover(cp1, cp2)
            Q = f(child, minQos, maxQos, constraints, weightList)
            if Q > fitnessList[i]:
                fitnessList[i] = Q
                solutions[i] = child
                limit[i] = 0
            else:
                limit[i] += 1

        # Probability update
        for i in range(SN):
            s = solutions[i]
            probabilityList[i] = fitnessList[i] / sum(fitnessList)

        # onlooker bees phase
        for bee in range(onlookers):
            i = randint(0, SN - 1)
            if probabilityList[i] > random.random():
                cp = solutions[i]  # cp is a composition plan
                # choose randomly a service to mutate
                service = cp.G.nodes[randint(0, cp.G.number_of_nodes() - 1)]["service"]
                # new service index
                neighbors = getNeighbors(service, candidates)
                N = len(neighbors)
                # mutation
                if random.random() <= 0.7 :
                    cp.mutate(neighbors[(N - 1) // itera])
                    Q = f(cp, minQos, maxQos, constraints, weightList)
                    if Q > fitnessList[i]:
                        fitnessList[i] = Q
                        limit[i] = 0
                    else:
                        cp.mutate(service)  # mutation reset
                        limit[i] += 1

        # scout bees phase
        for bee in range(scouts):
            i = randint(0, SN - 1)
            if limit[i] == SQ:  # scouts bee condition verified
                if itera >= CP:
                    cp = solutions[i]
                    # choose randomly a service to mutate
                    service = cp.G.nodes[randint(0, cp.G.number_of_nodes() - 1)]["service"]
                    # new service index
                    neighbors = getNeighbors(service, candidates)
                    # mutation
                    cp.mutate(neighbors[1])
                    Q = f(cp, minQos, maxQos, constraints, weightList)
                    fitnessList[i] = Q
                    limit[i] = 0

                else:
                    # Scouting
                    minIndex = fitnessList.index(min(fitnessList))  # lowest fitness
                    cp = CompositionPlan(actGraph, candidates)
                    solutions[minIndex] = cp
                    fitnessList[minIndex] = f(cp, minQos, maxQos, constraints, weightList)
                    limit[minIndex] = 0

        # the best of each iteration
        best_itera = max(fitnessList)
        if best_itera > best_fit :
            best = solutions[fitnessList.index(best_itera)]
            best_fit = best_itera

    return best , best_fit

In [0]:
import numpy as np
import time
import matplotlib.pyplot as plt
import csv
import random
def minMaxQos(num_act, actGraph):
    servicesMin = []
    servicesMax = []
    listQos = []
    for i in range(num_act):
        servicesMin.append([Service(i, 0.1, 0.7, 0.9, 0.1, matching=1)])
        servicesMax.append([Service(i, 5, 0.95, 0.99, 3, matching=1)])

    cpMin = CompositionPlan(actGraph, servicesMin)
    cpMax = CompositionPlan(actGraph, servicesMax)

    k = [cpMin.cpResponseTime(), cpMin.cpPrice(),cpMin.cpAvailability(), cpMin.cpReliability()]
    L = ['responseTime', 'price', 'availability', 'reliability']
    minQos = {i: j for i, j in zip(L, k)}

    k = [cpMax.cpResponseTime(), cpMax.cpPrice(), cpMax.cpAvailability(), cpMax.cpReliability()]
    L = ['responseTime', 'price', 'availability', 'reliability']
    maxQos = {i: j for i, j in zip(L, k)}

    return minQos, maxQos


def generateActGraph(num_act):       # Sequential
    return [[i, i + 1, 0] for i in range(num_act - 1)]


def generateCandidates(num_act, num_candidates):
    candidates = []
    for i in range(num_act):
        s = []
        for j in range(num_candidates):
            responseTime = np.random.uniform(0.1, 5, 1)[0]
            price = np.random.uniform(0.1, 3, 1)[0]
            availability = np.random.uniform(0.9, 0.99, 1)[0]
            reliability = np.random.uniform(0.7, 0.95, 1)[0]
            s.append(Service(i, responseTime, reliability, availability, price, matching=1))
        candidates.append(s)
    return candidates


def test(num_act, num_candidates,sn,mcn,sq, constraints, weightList):

    x = str((num_act, num_candidates,sn,mcn,sq))
    y = []
    z = []

    actGraph = generateActGraph(num_act)

    print("Generating random candidates for {} ...".format(x))

    candidates = generateCandidates(num_act, num_candidates)

    print("Done !")

    # minQos and maxQos definition

    minQos, maxQos = minMaxQos(num_act, actGraph)


    # optimal fitness
    opt = 2.0


    # Algorithm execution

    print("Executing Algorithm ")
    start_time = time.time()
    _ , fit = ABCgenetic(actGraph, candidates, workers=sn//2, onlookers=sn//2, scouts=sn//2, SQ=sq, MCN=mcn, SN=sn, minQos=minQos, maxQos=maxQos, constraints=constraints, weightList=weightList)
    rt = time.time() - start_time

    with open('datasetcolab.csv', mode='a') as file:
        file_writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        file_writer.writerow([num_act , num_candidates , sq , mcn , sn , fit / opt , rt])



    print('\nDone !')


# main
constraints = {'responseTime': 10, 'price': 1000, 'availability': 10, 'reliability': 10}
weightList = [0.25, 0.25, 0.25, 0.25]

for i in range(1,11) :
    for j in range(1,50):
        num_candidates = random.randint(5,500)
        for k in range(1,6):
            sn = random.randint(100,200*k)
            mcn = random.randint(100,200*k)
            sq = random.randint(10,10*k)
            test(i*5, num_candidates,sn,mcn,sq, constraints, weightList)

In [0]:
files.download('datasetcolab.csv')